In [7]:
import torch
import torch.nn as nn
from torch import optim
import numpy as np

import gym
import random

from model import DuelingQFunc
from ReplayMemory import ReplayMemory

lr = 1e-3
gamma = 0.95
epsilon = 0.3
batch_size = 32
initial_exploration = 500

qf = DuelingQFunc()
target_qf = DuelingQFunc()
#model.state_dict():モデルの学習パラメータをとってきている
target_qf.load_state_dict(qf.state_dict())

optimizer = optim.Adam(qf.parameters(), lr = lr)

criterion = nn.MSELoss()

memory = ReplayMemory()

env = gym.make('CartPole-v0')
obs_size =env.observation_space.shape[0]
action_size = env.action_space.n

total_step = 0
ten_step = 0

for episode in range(200):
    done = False
    obs = env.reset()
    sum_reward = 0
    step = 0
    
    while not done:
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
            action = qf.select_action(obs)
        epsilon -= 1e-4
        if epsilon < 0:
            epsilon = 0
            
        next_obs, reward, done, _ = env.step(action)
        
        terminal = 0
        reward = 0
        if done:
            terminal = 1
            if not step >= 195:
                reward = -1
        sum_reward += reward
        
        memory.add(obs, action, reward, next_obs, terminal)
        obs = next_obs.copy()
        
        step += 1
        total_step += 1
        if total_step < initial_exploration:
            continue
            
        batch = memory.sample()
        
        #各サンプルにおける状態行動の値を取ってくる
        q_value = qf(batch['obs']).gather(1, batch['actions'])
        
        #サンプルごとの処理を同時に行う
        with torch.no_grad():
            #Q-networkにおける最大値のインデックスを取ってくる
            max_next_q_value_index = qf(batch['next_obs']).max(dim = 1, keepdim = True)[1]
            #target-Q-network内の、対応する行動のインデックスにおける価値関数の値を取ってくる
            next_q_value = target_qf(batch['next_obs']).gather(1, max_next_q_value_index)
            #目的とする値の導出
            target_q_value = batch['rewards'] + gamma * next_q_value * (1 - batch['terminates'])
        
        #誤差の計算
        loss = criterion(q_value, target_q_value)
        #勾配を0にリセットする
        optimizer.zero_grad()
        #逆誤差伝搬を計算する
        loss.backward()
        #勾配を更新する
        optimizer.step()
        
        if total_step % 10 == 0:
            #targetネットワークの更新
            target_qf.load_state_dict(qf.state_dict())
            
    ten_step += step
    if episode % 10 == 0:
        print('episode:',episode, 'return:', ten_step / 10.0, 'epsilon:', epsilon)
        ten_step = 0

episode: 0 return: 0.8 epsilon: 0.2992000000000001
episode: 10 return: 10.7 epsilon: 0.28850000000000126
episode: 20 return: 10.9 epsilon: 0.27760000000000246
episode: 30 return: 10.4 epsilon: 0.2672000000000036
episode: 40 return: 11.0 epsilon: 0.2562000000000048
episode: 50 return: 11.4 epsilon: 0.24480000000000607
episode: 60 return: 18.0 epsilon: 0.22680000000000805
episode: 70 return: 44.8 epsilon: 0.18200000000001298
episode: 80 return: 112.2 epsilon: 0.06980000000001768
episode: 90 return: 163.5 epsilon: 0
episode: 100 return: 186.4 epsilon: 0
episode: 110 return: 190.2 epsilon: 0
episode: 120 return: 199.4 epsilon: 0
episode: 130 return: 197.5 epsilon: 0
episode: 140 return: 200.0 epsilon: 0
episode: 150 return: 190.7 epsilon: 0
episode: 160 return: 174.5 epsilon: 0
episode: 170 return: 191.1 epsilon: 0
episode: 180 return: 166.1 epsilon: 0
episode: 190 return: 172.2 epsilon: 0
